# Escenarios experimentales Internetworking
Presentado por: Jonathan Toapanta
Fecha: 23/02/2023


REPOSITORIO : https://github.com/jonathan-elian-toapanta/INTERNETWORKING

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #Gives us Graphics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [43]:
# cargar datos y normalizar
# Load and preprocess the data
url = 'https://raw.githubusercontent.com/beespinosa1/Inter/main/Escenario2.csv'
df = pd.read_csv(url)
df = pd.read_csv(url,low_memory=False)
dataframe = pd.read_csv(url,low_memory=False)

In [44]:
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name] = pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value)    
    return df
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()

In [45]:
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys)-1]].copy()
x_normalised = dfNormalize(data_to_process)

In [46]:
y = dataframe['label']
change_labels = lambda x: 1 if x == 'nonTOR' else 0
y_normalised = dataframe['label'].apply(change_labels)


In [47]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_normalised, y_normalised, test_size=0.2, random_state=42)

# Reshape the data to fit the input shape of the CNN
x_train = x_train.values.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.values.reshape(x_test.shape[0], x_test.shape[1], 1)

# Convert the labels to categorical format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [48]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


In [49]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
# Train the model
history = model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test))


Epoch 1/50
38/38 [==============================] - 2s 17ms/step - loss: 0.2803 - accuracy: 0.9887 - val_loss: 0.0379 - val_accuracy: 0.9967
Epoch 2/50
38/38 [==============================] - 0s 12ms/step - loss: 0.0038 - accuracy: 0.9996 - val_loss: 0.0515 - val_accuracy: 0.9967
Epoch 3/50
38/38 [==============================] - 0s 11ms/step - loss: 0.0068 - accuracy: 0.9996 - val_loss: 0.0435 - val_accuracy: 0.9967
Epoch 4/50
38/38 [==============================] - 0s 11ms/step - loss: 0.0049 - accuracy: 0.9996 - val_loss: 0.0407 - val_accuracy: 0.9967
Epoch 5/50
38/38 [==============================] - 0s 11ms/step - loss: 0.0044 - accuracy: 0.9996 - val_loss: 0.0426 - val_accuracy: 0.9967
Epoch 6/50
38/38 [==============================] - 0s 12ms/step - loss: 0.0041 - accuracy: 0.9996 - val_loss: 0.0376 - val_accuracy: 0.9967
Epoch 7/50
38/38 [==============================] - 0s 11ms/step - loss: 0.0057 - accuracy: 0.9996 - val_loss: 0.0374 - val_accuracy: 0.9967
Epoch 8/50
38

In [51]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_bin = (y_pred > 0.5).astype(int)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    precision = precision_score(y_test, y_pred_bin, average='weighted')
    recall = recall_score(y_test, y_pred_bin, average='weighted')
    f1 = f1_score(y_test, y_pred_bin, average='weighted')
    cm = confusion_matrix(y_test.argmax(axis=1), y_pred_bin.argmax(axis=1))
    print("Confusion Matrix:\n", cm)
    print("Loss: {:.4f}\nAccuracy: {:.4f}\nPrecision: {:.4f}\nRecall: {:.4f}\nF1 Score: {:.4f}".format(loss, accuracy, precision, recall, f1))

In [52]:
evaluate_model(model, x_test, y_test)



19/19 [==============================] - 0s 2ms/step
Confusion Matrix:
 [[  0   2]
 [  0 598]]
Loss: 0.0394
Accuracy: 0.9967
Precision: 0.9933
Recall: 0.9967
F1 Score: 0.9950


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
